<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/himalayaoption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import unittest

# Helper to mimic TopLevelFixture - ensures settings are restored
class QuantLibTestCase(unittest.TestCase):
    def setUp(self):
        self.saved_settings = ql.SavedSettings()
        # Set a default evaluation date if tests rely on it,
        # otherwise, each test should set its own.
        # For this specific test, the C++ test uses Settings::instance().evaluationDate()
        # which defaults to today's date if not set otherwise by a fixture.
        # We'll set it to a fixed date for reproducibility, similar to how
        # test suites often operate.
        self.evaluation_date = ql.Date(15, ql.May, 2020) # Example fixed date
        ql.Settings.instance().evaluationDate = self.evaluation_date


    def tearDown(self):
        self.saved_settings = None # Restores settings

def flatRate(evaluationDate, rate, dayCounter):
    """Helper to create a flat yield term structure handle."""
    return ql.YieldTermStructureHandle(ql.FlatForward(evaluationDate, rate, dayCounter))

def flatVol(evaluationDate, vol, dayCounter):
    """Helper to create a flat black volatility term structure handle."""
    return ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(evaluationDate, ql.NullCalendar(), vol, dayCounter)
    )


class HimalayaOptionTests(QuantLibTestCase):

    def testCached(self):
        self.subTestName = "Testing Himalaya option against cached values..."
        # print(self.subTestName) # Optional: for verbose output

        today = ql.Settings.instance().evaluationDate # Uses the date set in setUp

        dc = ql.Actual360()
        fixingDates = []
        for i in range(5): # 5 fixing dates
            fixingDates.append(today + ql.Period(i * 90, ql.Days))

        strike = 101.0
        # HimalayaOption(const std::vector<Date>& fixingDates, Real strike = Null<Real>());
        # Python: HimalayaOption(fixingDates, strike)
        option = ql.HimalayaOption(fixingDates, strike)

        riskFreeRate = flatRate(today, 0.05, dc)

        # Create a list of 1D stochastic processes
        processes_list = [] # Python list to hold the processes

        # Process 0
        s0_0 = ql.RelinkableQuoteHandle(ql.SimpleQuote(100.0))
        q_0 = flatRate(today, 0.01, dc)
        vol_0 = flatVol(today, 0.30, dc)
        processes_list.append(ql.BlackScholesMertonProcess(s0_0, q_0, riskFreeRate, vol_0))

        # Process 1
        s0_1 = ql.RelinkableQuoteHandle(ql.SimpleQuote(110.0))
        q_1 = flatRate(today, 0.05, dc)
        vol_1 = flatVol(today, 0.35, dc)
        processes_list.append(ql.BlackScholesMertonProcess(s0_1, q_1, riskFreeRate, vol_1))

        # Process 2
        s0_2 = ql.RelinkableQuoteHandle(ql.SimpleQuote(90.0))
        q_2 = flatRate(today, 0.04, dc)
        vol_2 = flatVol(today, 0.25, dc)
        processes_list.append(ql.BlackScholesMertonProcess(s0_2, q_2, riskFreeRate, vol_2))

        # Process 3
        s0_3 = ql.RelinkableQuoteHandle(ql.SimpleQuote(105.0))
        q_3 = flatRate(today, 0.03, dc)
        vol_3 = flatVol(today, 0.20, dc)
        processes_list.append(ql.BlackScholesMertonProcess(s0_3, q_3, riskFreeRate, vol_3))

        # Correlation matrix
        # ql.Matrix(rows, columns)
        correlation = ql.Matrix(4, 4)
        correlation[0][0] = 1.00; correlation[0][1] = 0.50; correlation[0][2] = 0.30; correlation[0][3] = 0.10
        correlation[1][0] = 0.50; correlation[1][1] = 1.00; correlation[1][2] = 0.20; correlation[1][3] = 0.40
        correlation[2][0] = 0.30; correlation[2][1] = 0.20; correlation[2][2] = 1.00; correlation[2][3] = 0.60
        correlation[3][0] = 0.10; correlation[3][1] = 0.40; correlation[3][2] = 0.60; correlation[3][3] = 1.00

        # Create the StochasticProcessArray
        # StochasticProcessArray(const std::vector<ext::shared_ptr<StochasticProcess1D> >&,
        #                        const Matrix& correlation);
        multi_process = ql.StochasticProcessArray(processes_list, correlation)

        seed = 86421 # BigNatural in C++, int in Python is fine
        fixedSamples = 1023

        # MCHimalayaEngine<PseudoRandom> in C++
        # Python: MCHimalayaEngine(process, antitheticVariate=False, controlVariate=False,
        #                         requiredSamples=NullSize, requiredTolerance=NullReal,
        #                         maxSamples=NullSize, seed=0, Sobol=False)
        # The .withSamples() and .withSeed() are builder patterns not directly in Python constructor.
        # We pass them as arguments.

        # First pricing run with fixed samples
        engine_fixed_samples = ql.MCHimalayaEngine(
            multi_process,
            requiredSamples=fixedSamples,
            seed=seed,
            Sobol=False # For PseudoRandom
        )
        option.setPricingEngine(engine_fixed_samples)

        value = option.NPV()
        storedValue = 5.93632056
        tolerance_npv = 1.0e-8 # Tolerance for comparing NPV directly

        self.assertAlmostEqual(value, storedValue, delta=tolerance_npv,
                               msg=(f"Cached Himalaya NPV mismatch. "
                                    f"Calculated: {value:.10f}, Expected: {storedValue:.10f}"))

        # Second pricing run with absolute tolerance for error estimate
        # minimumTol is a C++ variable, not a parameter to the engine.
        # The tolerance for the engine is set based on the error estimate of the *previous* run.
        # C++: tolerance = option.errorEstimate();
        #      tolerance = std::min<Real>(tolerance/2.0, minimumTol*value);
        #      engine.withAbsoluteTolerance(tolerance)

        # Get error estimate from the first run (it's already calculated if NPV was called)
        error_estimate_first_run = option.errorEstimate()

        minimum_tol_for_accuracy_check = 1.0e-2 # from C++ test
        # Calculate the target tolerance for the second engine run
        target_accuracy_tolerance = min(error_estimate_first_run / 2.0, minimum_tol_for_accuracy_check * value)
        # Ensure target_accuracy_tolerance is positive if value or error_estimate_first_run is very small/zero
        if target_accuracy_tolerance <=0:
            target_accuracy_tolerance = 1e-5 # A small positive fallback

        engine_abs_tolerance = ql.MCHimalayaEngine(
            multi_process,
            requiredTolerance=target_accuracy_tolerance,
            seed=seed,
            Sobol=False # For PseudoRandom
        )
        option.setPricingEngine(engine_abs_tolerance)

        option.NPV() # This call triggers the calculation and error estimate update
        accuracy_reached = option.errorEstimate()

        self.assertLessEqual(accuracy_reached, target_accuracy_tolerance,
                              msg=(f"Himalaya engine failed to reach required accuracy. "
                                   f"Reached: {accuracy_reached:.10f}, Expected: {target_accuracy_tolerance:.10f}"))


if __name__ == '__main__':
    print("Running QuantLib-Python HimalayaOptionTests...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

Key Points of the Python Translation:
QuantLibTestCase: Inherits from unittest.TestCase and uses ql.SavedSettings() for managing QuantLib's global settings, similar to TopLevelFixture. An example evaluation_date is set in setUp for reproducibility.
Helper Functions (flatRate, flatVol): These are small utility functions to create YieldTermStructureHandle and BlackVolTermStructureHandle for flat rates and volatilities, making the main test code cleaner.
Date and Day Counter: ql.Date, ql.Actual360(), ql.Period are used as direct equivalents.
HimalayaOption: Instantiated directly with fixing dates and strike.
Processes:
ql.BlackScholesMertonProcess is used for each underlying.
Handles (ql.RelinkableQuoteHandle, ql.YieldTermStructureHandle, ql.BlackVolTermStructureHandle) are used for inputs.
The list of 1D processes is a standard Python list.
ql.StochasticProcessArray is used to combine the 1D processes with the correlation matrix.
Correlation Matrix: ql.Matrix is created and populated.
MCHimalayaEngine:
The C++ code uses a builder pattern (MakeMCHimalayaEngine<...>().withSamples().withSeed()). The Python ql.MCHimalayaEngine constructor takes these parameters directly (e.g., requiredSamples, seed, requiredTolerance).
Sobol=False is specified to match PseudoRandom from the C++ template.
NPV and Error Estimate: option.NPV() and option.errorEstimate() are used.
Assertions: self.assertAlmostEqual and self.assertLessEqual are used from the unittest module to check values and tolerances.
Tolerance Logic for Second Run: The C++ test calculates a tolerance for the second engine run based on the error estimate of the first run and a minimumTol. This logic is replicated in Python to set target_accuracy_tolerance. A small positive fallback for target_accuracy_tolerance is added to prevent issues if the calculated tolerance becomes non-positive.
